In [14]:
import numpy as np
from scipy.special import jv

In [18]:
def dbesj(x, alpha, n):
    """sequence of bessel functions where n is the number of elements of a 
    bessel function of J_(alpha + k -1)/x where k = 1...n
    """
    bessel_j = np.zeros(n+1)
    i = 0
    for k in range(1,n+1):
       bessel_j[i] = jv(alpha + k -1, x)
       i = i + 1
    y = sum([num for num in bessel_j if num== 0])
    return bessel_j, y

In [19]:
x = 500
alpha = 0

In [20]:
%timeit dbesj(x,alpha,100)

1000 loops, best of 3: 518 µs per loop


In [26]:
b = [10,9,8,7,6,5,4,3,2,1]
next(i[0] for i in enumerate(b) if i[1] < 5)

6

In [24]:
enumerate(b)